In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

BASE_URL = "https://unityleague.gg"
LEADERBOARD_URL = f"{BASE_URL}/ranking/eu2025/BE/?showAll=true"

def get_soup_from_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"[ERROR] Failed to fetch {url} - Status code: {response.status_code}")
        return None
    return BeautifulSoup(response.text, 'html.parser')

def extract_players(soup):
    print("[INFO] Extracting players from leaderboard...")
    players = []
    rows = soup.select("table#rankingTable tbody tr")
    print(f"[DEBUG] Found {len(rows)} player rows in leaderboard")
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 3:
            print("[WARN] Skipping malformed row with insufficient columns")
            continue
        rank = cols[0].get_text(strip=True)
        points = int(cols[1].get_text(strip=True))
        player_tag = cols[2].find("a")
        if not player_tag:
            print("[WARN] No player link found in row")
            continue
        name = player_tag.get_text(strip=True)
        profile_url = BASE_URL + player_tag['href']
        player_id = player_tag['href'].split('/')[2]
        players.append({
            "rank": rank,
            "points": points,
            "name": name,
            "profile_url": profile_url,
            "player_id": player_id
        })
    return players

def parse_player_profile(url):
    soup = get_soup_from_url(url)
    if not soup:
        return None

    # 🎯 Find the "Performance per format" table using its heading
    perf_heading = soup.find("h3", string=re.compile("Performance per format", re.I))
    if not perf_heading:
        print("[WARN] 'Performance per format' heading not found.")
        return None

    perf_table = perf_heading.find_next("table")
    if not perf_table:
        print("[WARN] No table found under 'Performance per format'")
        return None

    # Find Limited row
    rows = perf_table.select("tbody tr")
    limited_row = next((r for r in rows if "Limited" in r.get_text()), None)
    if not limited_row:
        print("[INFO] No Limited format data found, skipping player.")
        return None

    columns = limited_row.find_all("td")
    record_text = columns[1].text.strip()
    winrate = columns[2].text.strip()
    wins, losses, *_ = map(int, re.findall(r"\d+", record_text))
    print(f"[DEBUG] Limited Record: {record_text} -> Wins: {wins}, Losses: {losses}, Winrate: {winrate}")

    # 🎯 Extract Limited events only from event history
    try:
        events_section = soup.find("h2", string=re.compile("Event history", re.I)).find_next("table")
        event_rows = events_section.find_all("tr")[1:]
        limited_events = [r for r in event_rows if "Limited" in r.text and "draft" in r.text.lower()]
        print(f"[DEBUG] Found {len(limited_events)} Limited Draft events")
    except Exception as e:
        print(f"[ERROR] Failed to parse event history: {e}")
        limited_events = []

    return {
        "wins": wins,
        "losses": losses,
        "winrate": winrate,
        "limited_events": len(limited_events),
        "matches": wins + losses
    }

def main():
    print("[START] Scraping Unity League Limited Draft Leaderboard")
    leaderboard_soup = get_soup_from_url(LEADERBOARD_URL)
    if not leaderboard_soup:
        print("[FATAL] Could not load leaderboard page.")
        return

    players = extract_players(leaderboard_soup)
    results = []

    for player in players[0:40]:
        print(f"[INFO] Processing {player['name']} ({player['profile_url']})")
        try:
            stats = parse_player_profile(player['profile_url'])
            if stats:
                player.update(stats)
                results.append(player)
            else:
                print(f"[SKIP] No Limited results for {player['name']}")
        except Exception as e:
            print(f"[ERROR] Exception while processing {player['name']}: {e}")
        time.sleep(1)  # Be polite to the server

    print("\n[RESULTS] Players with Limited format stats:")
    for p in results:
        print(f"{p['rank']}. {p['name']} - {p['points']} pts - "
              f"{p['wins']}W/{p['losses']}L - {p['winrate']} over {p['limited_events']} Limited events")
    return results


if __name__ == "__main__":
    results = main()


[START] Scraping Unity League Limited Draft Leaderboard
[INFO] Fetching URL: https://unityleague.gg/ranking/eu2025/BE/?showAll=true
[SUCCESS] Fetched https://unityleague.gg/ranking/eu2025/BE/?showAll=true
[INFO] Extracting players from leaderboard...
[DEBUG] Found 304 player rows in leaderboard
[INFO] Processing Nicolas Prail (https://unityleague.gg/player/3681/)
[INFO] Parsing player profile: https://unityleague.gg/player/3681/
[INFO] Fetching URL: https://unityleague.gg/player/3681/
[SUCCESS] Fetched https://unityleague.gg/player/3681/
[INFO] No Limited format data found, skipping player.
[SKIP] No Limited results for Nicolas Prail
[INFO] Processing Ignacio J. Garcia S. (https://unityleague.gg/player/16227/)
[INFO] Parsing player profile: https://unityleague.gg/player/16227/
[INFO] Fetching URL: https://unityleague.gg/player/16227/
[SUCCESS] Fetched https://unityleague.gg/player/16227/
[INFO] No Limited format data found, skipping player.
[SKIP] No Limited results for Ignacio J. Garc

In [24]:

df = pd.DataFrame(results)

In [25]:
df 

,rank,points,name,profile_url,player_id,wins,losses,winrate,limited_events,matches
0,9,266,Aymeric Steinbach,https://unityleague.gg/player/11248/,11248,30,17,63.8%,13,47
1,10,246,Thomas Bijl,https://unityleague.gg/player/11219/,11219,18,13,58.1%,9,31
2,12,226,Thomas Decamp,https://unityleague.gg/player/11230/,11230,27,19,58.7%,10,46
3,16,189,"Maxime, dit le BG, Corchia",https://unityleague.gg/player/11211/,11211,7,0,100.0%,0,7
4,17,165,Niels Lucking,https://unityleague.gg/player/11255/,11255,12,5,70.6%,3,17
5,18,160,Emmanuel Fleau De Frôler,https://unityleague.gg/player/11215/,11215,8,3,72.7%,1,11
6,19,154,Ricardo R,https://unityleague.gg/player/11259/,11259,17,12,58.6%,7,29
7,21,146,Nicolas Sargos,https://unityleague.gg/player/11223/,11223,30,8,73.2%,11,38
8,22,127,Martin Alberty,https://unityleague.gg/player/11217/,11217,3,4,42.9%,0,7
9,23,122,Maciej Mrozowski,https://unityleague.gg/player/11224/,11224,25,14,61.0%,11,39
